In [1]:
import multiprocessing as mp

def sending_process(queue, numMsg):
    for i in range(numMsg):
        data = 10000000*[357]
        queue.put(data)

def receiving_process(queue):
    while True:
        data = queue.get()
        queue.task_done()
        print(queue.qsize())

In [2]:
queue = mp.JoinableQueue(1000)
proc1 = mp.Process(target=sending_process, args=(queue, 20))
proc2 = mp.Process(target=receiving_process, args=(queue,))
proc1.start()
proc2.start()

5
12
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0


Process Process-2:
KeyboardInterrupt
Traceback (most recent call last):
  File "/home/robert/anaconda3/envs/CU-PWFA/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/robert/anaconda3/envs/CU-PWFA/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-1-78abf61ecce2>", line 10, in receiving_process
    data = queue.get()
  File "/home/robert/anaconda3/envs/CU-PWFA/lib/python3.6/multiprocessing/queues.py", line 94, in get
    res = self._recv_bytes()
  File "/home/robert/anaconda3/envs/CU-PWFA/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/robert/anaconda3/envs/CU-PWFA/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/robert/anaconda3/envs/CU-PWFA/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, rem

In [3]:
import multiprocessing as mp
import PyCapture2 as pc2
import time

def sending_process(queue):
    bus = pc2.BusManager()
    cam = pc2.GigECamera()
    guid = bus.getCameraFromSerialNumber(17570564)
    cam.connect(guid)
    cam.startCapture(image_callback)
    time.sleep(1)
    cam.stopCapture()
    cam.disconnect()
    
def image_callback(image):
    #data = image.getData()
    data = 10000000*[327]
    queue.put(data)

def receiving_process(queue):
    while True:
        data = queue.get()
        queue.task_done()
        print(queue.qsize())

In [4]:
queue = mp.JoinableQueue(1000)
proc1 = mp.Process(target=sending_process, args=(queue,))
proc2 = mp.Process(target=receiving_process, args=(queue,))
proc1.start()
proc2.start()
# Calling queue.get() blocks forever even though the queue is not empty

3
8
8


Process Process-4:
Traceback (most recent call last):
  File "/home/robert/anaconda3/envs/CU-PWFA/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/robert/anaconda3/envs/CU-PWFA/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-11210a985eb9>", line 22, in receiving_process
    data = queue.get()
  File "/home/robert/anaconda3/envs/CU-PWFA/lib/python3.6/multiprocessing/queues.py", line 94, in get
    res = self._recv_bytes()
  File "/home/robert/anaconda3/envs/CU-PWFA/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/robert/anaconda3/envs/CU-PWFA/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/robert/anaconda3/envs/CU-PWFA/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardIn

In [5]:
print(queue.qsize())
# If queue.get() is not called at all until after stopCapture has executed,
# then the entire queue can be emptied as usual
# If queue.get() is called before stopCapture, an item is removed from the queue,
# but if queue.get() is called again after stopCapture it doesn't work and neither does stopCapture;
# however, this behavior isn't universally true, occasionally it still works

8


In [10]:
import multiprocessing as mp
import PyCapture2 as pc2
import time

def sending_process(queue):
    # Connect to the camera
    bus = pc2.BusManager()
    cam = pc2.GigECamera()
    guid = bus.getCameraFromSerialNumber(17570564)
    cam.connect(guid)
    # Record images for 1s
    cam.startCapture(image_callback)
    time.sleep(1)
    cam.stopCapture()
    cam.disconnect()
    
def image_callback(image):
    #data = image.getData()
    data = 10000000*[327]
    queue.put(data)

queue = mp.JoinableQueue(1000)
proc1 = mp.Process(target=sending_process, args=(queue,))
proc1.start()
data = queue.get()
queue.task_done()
print(queue.qsize())

10


In [12]:
data = queue.get()
queue.task_done()
print(queue.qsize())

KeyboardInterrupt: 

In [15]:
import multiprocessing as mp
import PyCapture2 as pc2
import time

def sending_process(queue, numImage):
    # Connect to the camera
    bus = pc2.BusManager()
    cam = pc2.GigECamera()
    guid = bus.getCameraFromSerialNumber(17570564)
    cam.connect(guid)
    # Record images for 1s
    cam.startCapture()
    for i in range(numImage):
        image = cam.retrieveBuffer()
        data = image.getData()
        queue.put(data)
    cam.stopCapture()
    cam.disconnect()
    
def receiving_process(queue):
    while True:
        data = queue.get()
        queue.task_done()
        print(queue.qsize())

queue = mp.JoinableQueue(1000)
proc1 = mp.Process(target=sending_process, args=(queue, 10))
proc2 = mp.Process(target=receiving_process, args=(queue,))
proc1.start()
proc2.start()
data = queue.get()
queue.task_done()
print(queue.qsize())

3
4
5
6
5
4
3
2
1
0


In [12]:
import threading
import multiprocessing as mp
import PyCapture2 as pc2
import time

def sending_process(queue):
    # Connect to the camera
    bus = pc2.BusManager()
    cam = pc2.GigECamera()
    guid = bus.getCameraFromSerialNumber(17570564)
    cam.connect(guid)
    lock = threading.Lock()
    # Record images for 1s
    cam.startCapture(image_callback, lock)
    time.sleep(1)
    lock.acquire()
    cam.stopCapture()
    lock.release()
    cam.disconnect()
    
def image_callback(image, lock):
    lock.acquire()
    #data = image.getData()
    data = 10000000*[327]
    queue.put(data)
    lock.release()

queue = mp.JoinableQueue(1000)
proc1 = mp.Process(target=sending_process, args=(queue,))
proc1.start()
data = queue.get()
queue.task_done()
print(queue.qsize())

10


In [17]:
data = queue.get()
queue.task_done()
print(queue.qsize())

13
